In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt # visualization

In [2]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data = np.array(data) #put pandas dataframe into numpy array
m, n = data.shape #m is amount of rows, n is amount of features + 1
np.random.shuffle(data) #randomly rearrange training examples

# Split data into training and validation datasets

data_validate = data[0:1000].T # validation set
Y_validate = data_validate[0] # labels
X_validate = data_validate[1:n] # features
X_validate = X_validate / 255

data_train = data[1000:m].T # training set
Y_train = data_train[0] # labels
X_train = data_train[1:n] #features
X_train = X_train / 255
_,m_train = X_train.shape

In [5]:
X_train[:, 0].shape # should output how many pixels in each image

(784,)

In [6]:
Y_train

array([1, 2, 9, ..., 2, 1, 6])

In [7]:
def init_params():
    W1 = np.random.randn(10, 784) - 0.5
    b1 = np.random.randn(10, 1) - 0.5
    W2 = np.random.randn(10, 10) - 0.5
    b2 = np.random.randn(10, 1) - 0.5
    return W1, b1, W2, b2

#define ReLU (recitified linear unit) activation function
def ReLU(Z):
    return np.maximum(0, Z) # if x > 0: y = x, else: y = 0

#define ReLU (recitified linear unit) activation function
def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z)) # returns output layer of probabilties (0-1)

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1 # weighted sum of input layer
    A1 = ReLU(Z1) # use ReLU activation function on weighted sum of input layer to get first hidden layer
    Z2 = W2.dot(A1) + b2 # weighted sum of input layer
    A2 = softmax(Z2) # use softmax activation function on weighted sum of first hidden layer to get output layer of probabilities
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1)) # Creates matrix of zeros, with dimensions m (number of examples) and n (possible output classes)
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0 #if Z > 0, derivative will be derivative of x (1 or TRUE), if Z <= 0, derivative will be derivative of 0 (0 or FALSE)

def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) + deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - dW1 * alpha
    b1 = b1 - db1 * alpha
    W2 = W2 - dW2 * alpha
    b2 = b2 - db2 * alpha
    return W1, b1, W2, b2

In [8]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size
    
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(A2), Y))
    return W1, b1, W2, b2

In [9]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, 0.05)

Iteration:  0
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.09665853658536586
Iteration:  10
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.09665853658536586
Iteration:  20
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.09665853658536586
Iteration:  30
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.09665853658536586
Iteration:  40
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.09680487804878049
Iteration:  50
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.09895121951219513
Iteration:  60
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.11436585365853659
Iteration:  70
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.14360975609756096
Iteration:  80
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.16107317073170732
Iteration:  90
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.16853658536585367
Iteration:  100
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.1725121951219512
Iteration:  110
[8 8 8 ... 8 8 8] [1 2 9 ... 2 1 6]
Accuracy:  0.17385365853658535
Iteration:  120
